In [1]:
import torch
from torch import nn,Tensor
from d2l import torch as d2l

In [2]:
def corr2d_multi_in(X,K):
  return sum(d2l.corr2d(x,k) for x,k in zip(X,K))

In [4]:
X = torch.tensor([[[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]],
               [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]])
K = torch.tensor([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])
corr2d_multi_in(X,K)

tensor([[ 56.,  72.],
        [104., 120.]])

多通道的卷积核$K$是一个$4D$的$Tensor$，形状为$c_o\times c_i\times k_h\times k_w$

In [12]:

def corr2d_multi_in_out(X,K):
  """从这个4d的tensor中取出一个3d的tensor对输入做卷积（分别与对应通道的矩阵做互相关操作之后求和）

  Args:
      X (_type_): 输入
      K (_type_): 4d卷积核

  Returns:
      _type_: _description_
  """
  return torch.stack([corr2d_multi_in(X,k) for k in K],0)

In [8]:
K = torch.stack((K,K+1,K+2),0)
K.shape

torch.Size([3, 2, 2, 2])

In [18]:
corr2d_multi_in_out(X,K)

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

In [19]:
def corr2d_multi_in_out_1x1(X:Tensor,K:Tensor):
  c_i,h,w  =  X.shape
  c_o = K.shape[0]
  X = X.reshape((c_i,h*w))
  K = K.reshape((c_o,c_i))
  Y = torch.matmul(K,X)
  return Y.reshape((c_o,h,w))

In [20]:
X = torch.normal(0,1,(3,3,3))
K = torch.normal(0,1,(2,3,1,1))
Y1 = corr2d_multi_in_out_1x1(X,K)
Y2 = corr2d_multi_in_out(X,K)
assert float(torch.abs(Y1-Y2).sum()) <  1e-6

## 总结
+ 输出通道数是卷积层的超参数。
+ 每个输入通道有独立的二维卷积核，所有通道结果相加得到一个输出通道结果。
+ 每个输出通道有独立的三维卷积核。